In [1]:
%load_ext autoreload
%autoreload 2

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import sys
from itertools import groupby
import numpy as np
import cv2
from tqdm.notebook import tqdm
tqdm.pandas()
import pandas as pd
import os
from glob import glob
from multiprocessing import Pool
import matplotlib.pyplot as plt
# import cupy as cp
import ast
from pathlib import Path
import pickle5 as pickle
import torch

import shutil
import sys
sys.path.append("../src/")

import util

from joblib import Parallel, delayed

from IPython.display import display, HTML
from sahi.model import Yolov5DetectionModel
from sahi.utils.cv import read_image
from sahi.utils.file import download_from_url
from sahi.predict import get_prediction, get_sliced_prediction, predict
from ensemble_boxes import weighted_boxes_fusion

import albumentations as A

from IPython.display import Image
from matplotlib import animation, rc
rc('animation', html='jshtml')

In [3]:
df_GT = pd.read_csv( "../input/train_with_added_GT.csv")
df_GT['new_added_pred_LB'] = df_GT['new_added_pred_LB'].apply(ast.literal_eval)
GT_sizes = np.concatenate(df_GT['new_added_pred_LB'].apply(lambda x: [p['width'] * p['height'] for p in x ]))
#GT_sizes

In [4]:
np.quantile(GT_sizes / (1280 * 720), 0.8)

0.001400607638888889

# Load Data

In [5]:
use_GT = False
cv_scheme = 'v2'
INPUT_DIR = Path("../../data/tensorflow-great-barrier-reef/")
if use_GT:
    df = pd.read_csv( "../input/train_with_added_GT.csv")
else:
    df = pd.read_csv("../input/train.csv")
if cv_scheme == "video_id":
    df['fold'] = df['video_id']
elif cv_scheme == "v2":
    folds = util.load_pickle("../input/fold_test_2.pkl")
    df["fold"] = df["sequence"].apply(lambda x: folds[x])
    
highFP_df = pd.read_csv('../input/df_highFPNoBB.csv')
df = pd.merge(df, highFP_df[['video_id',"video_frame","highFBNoBB"]], on=["video_id","video_frame"], how='left')
df["highFBNoBB"].fillna(False, inplace=True)

data_param = {'root_dir':INPUT_DIR,'label_dir':INPUT_DIR / "labels"}
df = df.progress_apply(lambda x: util.get_path(x, data_param, infer=True), axis=1)
if use_GT:
    df['annotations'] = df['new_annotations_LB'].progress_apply(lambda x: ast.literal_eval(x))
else:
    df['annotations'] = df['annotations'].progress_apply(lambda x: ast.literal_eval(x))
df["real_bbox"] = df["annotations"].apply(lambda annots: [list(annot.values()) for annot in annots])
df['num_bbox'] = df['annotations'].progress_apply(lambda x: len(x))
data = (df.num_bbox>0).value_counts(normalize=True)*100
print(f"No BBox: {data[0]:0.2f}% | With BBox: {data[1]:0.2f}%")

  0%|          | 0/23501 [00:00<?, ?it/s]

  0%|          | 0/23501 [00:00<?, ?it/s]

  0%|          | 0/23501 [00:00<?, ?it/s]

No BBox: 79.07% | With BBox: 20.93%


In [6]:
df.groupby("video_id")['num_bbox'].sum()

video_id
0    3065
1    6384
2    2449
Name: num_bbox, dtype: int64

In [7]:
def add_df(name, from_col=None, to_col=None):
    path = "./oof_pred/" + name
    df_2 = util.load_pickle(path)
    if from_col is not None:
        df_2 = df_2.rename(columns = {from_col:to_col})
    for col in df_2.columns:
        if col not in df.columns:
            new_col = col
            if "_fold1" not in new_col and "pred" in new_col:
                new_col = new_col + "_fold1"
            df.loc[df.query('fold==1').index,new_col] = df_2.loc[df_2.query('fold==1').index,col].values
#add_df("pred_0207_yolov5m6_B_LS02_newGT_CV_video1.pkl")
add_df("pred_best_s6_3600_GT.pkl")
#add_df("img6000pred_0209_m6_B_LS02_newLGBT.pkl", "pred_0209_m6_B_LS02_newLGBT","pred_0209_m6_B_LS02_newLGBT_img6000")
add_df("pred_0209_m6_B_LS02_newLGBT.pkl")
#add_df("best_s6_3600_GT.pkl")

add_df("pred_0207_m6_B_LS02_newGT_clahe.pkl")
#add_df("pred_0208_s6_B_LS02_newGT_clahe.pkl")
#add_df("pred_0208_s6_B_LS02_newGT_clahe.pkl")
add_df('pred_0208_m_TPH_1900_B_LS02_newGT.pkl')            
add_df('pred_0209_m6_B_LS02_LGBT_newP.pkl')            
add_df('pred_oof_pred_r27-0210-yolov5m6-2400_fold1_img3120.pkl')
add_df('pred_0209_s6_B_LS02_newGT_img4096_valid.pkl')
#add_df('pred_CV3_0128.pkl')
#add_df('pred_CV3_s1800_0129.pkl')
# add_df('pred_CV3_s1800_e11_0129.pkl')
#add_df('pred_CV3_s1280_clahe_0129.pkl')
#add_df('pred_CV3_s3600_e11_0129.pkl', 'pred_0129_yolov5l_v3_highFP_1800_e11_fold1', 'pred_0129_yolov5l_v3_highFP_3600_e11_fold1')
#add_df('pred_CV3_s1800_clahe_0129.pkl')
#add_df('pred_CV3_s2700_clahe_0129.pkl', 'pred_0129_yolov5l_v3_1800_clahe_fold1', 'pred_0129_yolov5l_v3_2700_clahe_fold1')
# add_df('pred_CV3_m6_s2300_clahe_0130.pkl')
# add_df('pred_CV3_0130_exps.pkl')
# add_df('pred_CV3_0131_exps.pkl')

#add_df('pred_CV3_0203_sliced.pkl')
#add_df('pred_0130_yolov5m6_noClahe_2300_IOU07.pkl', 'pred_0130_yolov5m6_noClahe_2300_fold1', 'pred_0130_yolov5m6_noClahe_2300_IOU07_fold1')
#add_df('pred_CV3_0204_x6_1320.pkl')
# add_df('pred_0205_m_TPH_1900_B.pkl')
#add_df('pred_0205_yolov5s6_HA.pkl')
# add_df('pred_0205_yolov5m6_B.pkl')
# add_df('pred_0204_yolov5s6_B.pkl')
# add_df('pred_0205_m_P7_2300_B.pkl')
# add_df('pred_0205_m_TPH_1900_B_LR1e3.pkl')
# add_df('pred_0205_yolov5m6_B_LS02.pkl')
# add_df('pred_oof_pred_r22-0206-yolov5x6-1920_fold1_img2496.pkl')
# add_df('pred_0206_yolov5m6_B_LS01.pkl')
# add_df('pred_0206_yolov5m6_B_LS06.pkl')
# add_df('pred_0207_yolov5s6_B_LS02_newGT.pkl')
# add_df('pred_0207_yolov5m6_B_LS02_newGT.pkl')
# add_df('pred_0207_yolov5m6_B_LS02_newGT_clahe.pkl')
# add_df('pred_0207_m_TPH_1900_B_newGT.pkl')



In [8]:
df_oof = df.query("fold == 1").copy()

In [10]:
def get_predict(label_dir, image_id, conf_thr=0.01):
    label_path = label_dir + image_id + '.txt'
    #print(label_path)
    try:
        with open(label_path, 'r') as f:
            pred_string = f.read().splitlines()
        pred_bboxes = []
        for pred in pred_string:
            bbox = list(map(lambda i: float(i), pred.split(' ')))
            pred_bboxes.append(bbox[1:])
    except:
        # print(label_path)
        return [], []
        
    if len(pred_bboxes) == 0:
        return [], []
    pred_bboxes = np.array(pred_bboxes)
    #print(pred_bboxes[0])
    pred_bboxes[:, [0, 2]] = pred_bboxes[:, [0, 2]] * 1280
    pred_bboxes[:, [1, 3]] = pred_bboxes[:, [1, 3]] * 720
    pred_bboxes[:, [0, 1]] = pred_bboxes[:, [0, 1]] - pred_bboxes[:, [2, 3]] / 2
    pred_bboxes = pred_bboxes[pred_bboxes[:, 4] > conf_thr]
    return pred_bboxes[:, :4], pred_bboxes[:, 4]

def get_pred(label_dir, video_id, video_frame):
    bbox, conf = get_predict(label_dir, "{}-{}".format(video_id, video_frame))
    if len(bbox) > 0:
        bbox = bbox.tolist()
        conf = conf.tolist()
    return [[conf[i]] + bbox[i] for i in range(len(conf))]

In [11]:
df_oof['pred_GT3_4680_fold1'] = df_oof.apply(lambda x:  get_pred("./oof_pred/labels_GT3_4680/", x['video_id'], x['video_frame']), axis=1)
#df_oof['pred_GT_4680_fold1'] = df_oof.apply(lambda x:  get_pred("./oof_pred/labels_GT_4680/", x['video_id'], x['video_frame']), axis=1)

In [12]:
df_oof.columns

Index(['video_id', 'sequence', 'video_frame', 'sequence_frame', 'image_id',
       'annotations', 'fold', 'highFBNoBB', 'old_image_path', 'image_path',
       'label_path', 'real_bbox', 'num_bbox', 'train',
       'pred_best_s6_3600_GT_fold1', 'pred_0209_m6_B_LS02_newLGBT_fold1',
       'pred_0207_m6_B_LS02_newGT_clahe_fold1',
       'pred_0208_s6_B_LS02_newGT_fold1',
       'pred_0208_m_TPH_1900_B_LS02_newGT_fold1',
       'pred_0209_m6_B_LS02_LGBT_newP_fold1',
       'pred_r27-0210-yolov5m6-2400_fold1',
       'pred_0209_s6_B_LS02_newGT_img4096_valid_fold1', 'pred_GT3_4680_fold1'],
      dtype='object')

## mmdetection pred

In [13]:
# mm_model_versions = ["0202_frcnn_1600_highFP_e8"]
# for mm_v in mm_model_versions:
#     pred = util.load_pickle(f"../output/{mm_v}/test_result.pkl")
#     pred_new = []
#     for p in pred:
#         p_sub = p[0]
#         p_sub = p_sub[:,[4,0,1,2,3]]
#         p_sub[:,3:] = p_sub[:,3:] - p_sub[:,1:3]
#         pred_new.append(p_sub)
#     df_oof["pred_"+mm_v + "_fold1"] = pred_new

## All Model

In [15]:
model_versions = ["_".join(col.split("_")[1:-1]) for col in df_oof.columns if "_fold1" in col]
model_versions

['best_s6_3600_GT',
 '0209_m6_B_LS02_newLGBT',
 '0207_m6_B_LS02_newGT_clahe',
 '0208_s6_B_LS02_newGT',
 '0208_m_TPH_1900_B_LS02_newGT',
 '0209_m6_B_LS02_LGBT_newP',
 'r27-0210-yolov5m6-2400',
 '0209_s6_B_LS02_newGT_img4096_valid',
 'GT3_4680']

# Pick Conf Thresh

In [16]:
gt_bboxs_list = []
for index in df_oof.index:
    gt_bboxs_list.append(np.array(df_oof.loc[index, "real_bbox"]))

In [17]:
conf_ths = [5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55]
#conf_ths = np.arange(5, 20)
#conf_ths = np.arange(15, 25)
#conf_ths = np.arange(20, 40)
#conf_ths = np.arange(30, 40)
#conf_ths = [30]

#conf_ths = [12, 16, 18, 20, 22, 24]
#conf_ths = [6, 8, 10, 12, 14]
#conf_ths = [30, 32, 35, 38, 45, 50, 55]
iou_ths = np.arange(0.3, 0.85, 0.05)
fold = 1
check_models = ['GT3_4680']
for model_version in model_versions:
    if not model_version in check_models:
        continue
    pred_col = "pred_" + model_version + f"_fold{fold}"
    for conf_th in conf_ths:
        conf_th /= 100
        prd_bboxs_list = []
        for index in df_oof.index:
            prd_bboxs = df_oof.loc[index, pred_col]
            prd_bboxs = [p for p in prd_bboxs if p[0] >= conf_th]
            prd_bboxs_list.append(np.array(prd_bboxs))
        f2_dict = util.calc_f2_score(gt_bboxs_list, prd_bboxs_list, verbose=False)    
        print("model {}, conf_th {:.2f}, f2 {:.3f}, precision {:.3f}, recall {:.3f}".format(model_version, conf_th, f2_dict['f2'], f2_dict['P'], f2_dict['R']))

model GT3_4680, conf_th 0.05, f2 0.616, precision 0.336, recall 0.779
model GT3_4680, conf_th 0.10, f2 0.669, precision 0.445, recall 0.765
model GT3_4680, conf_th 0.15, f2 0.692, precision 0.517, recall 0.756
model GT3_4680, conf_th 0.20, f2 0.705, precision 0.571, recall 0.749
model GT3_4680, conf_th 0.25, f2 0.710, precision 0.611, recall 0.740
model GT3_4680, conf_th 0.30, f2 0.715, precision 0.646, recall 0.734
model GT3_4680, conf_th 0.35, f2 0.712, precision 0.673, recall 0.722
model GT3_4680, conf_th 0.40, f2 0.705, precision 0.698, recall 0.707
model GT3_4680, conf_th 0.45, f2 0.700, precision 0.725, recall 0.694
model GT3_4680, conf_th 0.50, f2 0.693, precision 0.751, recall 0.680
model GT3_4680, conf_th 0.55, f2 0.686, precision 0.779, recall 0.666


In [18]:
best_conf_dict = {
    '0209_m6_B_LS02_newLGBT':0.1,
    "0209_m6_B_LS02_newLGBT_img6000":0.15,
    '0207_m6_B_LS02_newGT_clahe': 0.25,
    '0208_s6_B_LS02_newGT': 0.30,
    '0208_s6_B_LS02_newGT_clahe': 0.30,
    '0208_m_TPH_1900_B_LS02_newGT':0.30,
    '0207_yolov5m6_B_LS02_newGT_CV_video1': 0.18,
    'best_s6_3600_GT':0.3, 
    '0209_m6_B_LS02_LGBT_newP': 0.20,
    'r27-0210-yolov5m6-2400': 0.27,
    '0209_s6_B_LS02_newGT_img4096_valid': 0.35,
    'GT3_4680': 0.33,
    'GT_4680': 0.33,
    
    '0128_yolov5l_v3': 0.25, #0.649
    "0128_yolov5l_v3_highFP": 0.25, #642
    "0129_yolov5l_v3_highFP_1800": 0.25, #645
    "0129_yolov5l_v3_highFP_1800_e11": 0.16,
    "0129_yolov5l_v3_highFP_clahe": 0.19,
    '0129_yolov5l_v3_highFP_3600_e11': 0.25,
    '0129_yolov5l_v3_1800_clahe': 0.15,
    '0129_yolov5l_v3_2700_clahe': 0.2,
    '0130_yolov5m6_2300': 0.27, # 660
    '0130_yolov5s6_3100': 0.31, # 646
    "0130_yolov5m6_noHighFP_2300": .40, # 655
    "0130_yolov5m6_noClahe_2300": .23, # 678
    "0130_yolov5m6_2300_sheep": .11, #669
    "0131_yolov5s6_3100_noClahe_sheep_whole": .29, #673
    '0131_frcnn_1600': 0.28,
    '0201_frcnn_1600_highFP' : 0.40,
    '0202_frcnn_1600_highFP_e8': 0.34, #621
    '0202_vf_1700_highFP_e11': 0.4,
    '0203_m6_1200_sliced_S720_ma025_lr5e4': 0.44, #666
    '0203_m6_1000_sliced_S400x711_ma025': 0.38,
    '0203_m6_1200_sliced_S720_ma025_lr5e4_AUG':0.53,
    '0203_yolov5x6_1320':0.31, 
    '0205_m_TPH_1900_B':0.27, # 660
    "0205_yolov5s6_HA":0.21, #666
    '0205_yolov5m6_B': 0.17, #688
    '0204_yolov5s6_B': 0.20, #683
    '0205_yolov5s6_MA': 0.23, #664
    '0205_m_P7_2300_B': 0.34, #659
    '0205_m_TPH_1900_B_LR1e3': 0.36, # 678
    '0205_yolov5m6_B_LS02': 0.3, # 687
    'r22-0206-yolov5x6-1920':0.2,
    "0206_yolov5m6_B_LS01": 0.15, # 671
    "0206_yolov5m6_B_LS06": 0.20, # 669
    '0207_yolov5s6_B_LS02_newGT':0.20, # 663
    '0207_yolov5m6_B_LS02_newGT':0.38, # 680,
    '0207_yolov5m6_B_LS02_newGT_clahe':0.33, # 682
    '0207_m_TPH_1900_B_newGT':0.21,#661
} 

In [19]:
pd.set_option('display.max_columns', 40)
f_cols = ["f2"] + ["f2_" + str(round(th,3)) for th in np.arange(0.3, 0.85, 0.05)]
P_cols = ["P"] + ["P_" + str(round(th,3)) for th in np.arange(0.3, 0.85, 0.05)]
R_cols = ["R"] + ["R_" + str(round(th,3)) for th in np.arange(0.3, 0.85, 0.05)]

In [21]:
df_perf_dict = {model_version: pd.DataFrame(columns=["all"] + [str(round(th,3)) for th in np.arange(0.3, 0.85, 0.05)]).copy() for model_version in model_versions}
fold = 1
f2_score = {}
P_score = {}
R_score = {}
f2_specific_keys = ['f2_0.3', 'f2_0.35', 'f2_0.4', 'f2_0.45', 'f2_0.5', 'f2_0.55', 'f2_0.6', 'f2_0.65', 'f2_0.7', 'f2_0.75', 'f2_0.8']
f2_specific_score = {k:{} for k in f2_specific_keys}
for model_version in model_versions:
    pred_col = "pred_" + model_version + f"_fold{fold}"
    conf_th = best_conf_dict[model_version]
    prd_bboxs_list = []
    for index in df_oof.index:
        prd_bboxs = df_oof.loc[index, pred_col]
        prd_bboxs = [p for p in prd_bboxs if p[0] >= conf_th]
        prd_bboxs_list.append(np.array(prd_bboxs))
    f2_dict = util.calc_f2_score(gt_bboxs_list, prd_bboxs_list, verbose=False)    
    df_perf_dict[model_version].loc["f2",:] = [f2_dict[col] for col in f_cols]
    df_perf_dict[model_version].loc["P",:] = [f2_dict[col] for col in P_cols]
    df_perf_dict[model_version].loc["R",:] = [f2_dict[col] for col in R_cols]
    f2_score[model_version] = f2_dict['f2']
    P_score[model_version] = f2_dict['P']
    R_score[model_version] = f2_dict['R']
    for k in f2_specific_keys:
        f2_specific_score[k][model_version] = f2_dict[k]
    print("model {}, conf_th {:.2f}, f2 {:.3f}, precision {:.3f}, recall {:.3f}".format(model_version, conf_th, f2_dict['f2'], f2_dict['P'], f2_dict['R']))
    display(df_perf_dict[model_version].astype(float).style.format("{:.3f}").background_gradient(axis=1))

model best_s6_3600_GT, conf_th 0.30, f2 0.702, precision 0.636, recall 0.720


,all,0.3,0.35,0.4,0.45,0.5,0.55,0.6,0.65,0.7,0.75,0.8
f2,0.702,0.817,0.817,0.816,0.813,0.808,0.794,0.762,0.712,0.605,0.475,0.298
P,0.636,0.740,0.740,0.739,0.737,0.732,0.719,0.690,0.645,0.548,0.431,0.270
R,0.720,0.839,0.839,0.838,0.835,0.830,0.815,0.782,0.731,0.621,0.488,0.306


model 0209_m6_B_LS02_newLGBT, conf_th 0.10, f2 0.691, precision 0.648, recall 0.702


,all,0.3,0.35,0.4,0.45,0.5,0.55,0.6,0.65,0.7,0.75,0.8
f2,0.691,0.794,0.794,0.793,0.793,0.791,0.780,0.756,0.705,0.604,0.473,0.313
P,0.648,0.746,0.745,0.745,0.744,0.742,0.732,0.710,0.662,0.567,0.444,0.294
R,0.702,0.808,0.807,0.806,0.806,0.804,0.793,0.769,0.717,0.614,0.481,0.318


model 0207_m6_B_LS02_newGT_clahe, conf_th 0.25, f2 0.695, precision 0.667, recall 0.703


...

,all,0.3,0.35,0.4,0.45,0.5,0.55,0.6,0.65,0.7,0.75,0.8
f2,0.704,0.815,0.815,0.815,0.812,0.804,0.790,0.764,0.711,0.617,0.481,0.318
P,0.671,0.777,0.777,0.777,0.774,0.767,0.753,0.729,0.678,0.588,0.458,0.303
R,0.713,0.825,0.825,0.825,0.822,0.814,0.800,0.774,0.720,0.624,0.487,0.322


model 0209_s6_B_LS02_newGT_img4096_valid, conf_th 0.35, f2 0.692, precision 0.732, recall 0.682


,all,0.3,0.35,0.4,0.45,0.5,0.55,0.6,0.65,0.7,0.75,0.8
f2,0.692,0.804,0.803,0.803,0.801,0.797,0.784,0.753,0.700,0.598,0.472,0.293
P,0.732,0.851,0.850,0.850,0.848,0.844,0.830,0.798,0.741,0.633,0.499,0.310
R,0.682,0.793,0.792,0.792,0.790,0.787,0.773,0.743,0.691,0.590,0.465,0.289


model GT3_4680, conf_th 0.33, f2 0.717, precision 0.662, recall 0.732


,all,0.3,0.35,0.4,0.45,0.5,0.55,0.6,0.65,0.7,0.75,0.8
f2,0.717,0.824,0.824,0.824,0.824,0.821,0.810,0.782,0.733,0.638,0.486,0.319
P,0.662,0.761,0.761,0.761,0.761,0.758,0.748,0.722,0.677,0.590,0.449,0.295
R,0.732,0.841,0.841,0.841,0.841,0.838,0.827,0.798,0.748,0.652,0.496,0.326


In [22]:
f2_score_df = pd.DataFrame.from_dict(f2_score,orient='index',columns=["f2"])
P_score_df = pd.DataFrame.from_dict(P_score,orient='index',columns=["P"])
R_score_df = pd.DataFrame.from_dict(R_score,orient='index',columns=["R"])
f2_s_df = []
for k in f2_specific_keys:
    f2_s_df.append(pd.DataFrame.from_dict(f2_specific_score[k],orient='index',columns=[k]))
score_df = pd.concat([f2_score_df, P_score_df, R_score_df] + f2_s_df, axis=1)

score_df.style.format("{:.3f}").background_gradient(axis=0)

,f2,P,R,f2_0.3,f2_0.35,f2_0.4,f2_0.45,f2_0.5,f2_0.55,f2_0.6,f2_0.65,f2_0.7,f2_0.75,f2_0.8
best_s6_3600_GT,0.702,0.636,0.720,0.817,0.817,0.816,0.813,0.808,0.794,0.762,0.712,0.605,0.475,0.298
0209_m6_B_LS02_newLGBT,0.691,0.648,0.702,0.794,0.794,0.793,0.793,0.791,0.780,0.756,0.705,0.604,0.473,0.313
0207_m6_B_LS02_newGT_clahe,0.695,0.667,0.703,0.801,0.801,0.799,0.798,0.795,0.785,0.759,0.709,0.611,0.472,0.319
0208_s6_B_LS02_newGT,0.666,0.636,0.674,0.772,0.772,0.772,0.772,0.769,0.753,0.721,0.667,0.587,0.452,0.292
0208_m_TPH_1900_B_LS02_newGT,0.640,0.621,0.644,0.736,0.736,0.736,0.736,0.735,0.729,0.704,0.646,0.559,0.438,0.282
0209_m6_B_LS02_LGBT_newP,0.703,0.683,0.708,0.806,0.806,0.806,0.806,0.802,0.789,0.764,0.710,0.631,0.493,0.320
r27-0210-yolov5m6-2400,0.704,0.671,0.713,0.815,0.815,0.815,0.812,0.804,0.790,0.764,0.711,0.617,0.481,0.318
0209_s6_B_LS02_newGT_img4096_valid,0.692,0.732,0.682,0.804,0.803,0.803,0.801,0.797,0.784,0.753,0.700,0.598,0.472,0.293
GT3_4680,0.717,0.662,0.732,0.824,0.824,0.824,0.824,0.821,0.810,0.782,0.733,0.638,0.486,0.319


# Tracking

In [23]:
from norfair import Detection, Tracker

# Helper to convert bbox in format [x_min, y_min, x_max, y_max, score] to norfair.Detection class
def to_norfair(detects, frame_id):
    result = []
    for x_min, y_min, x_max, y_max, score in detects:
        xc, yc = (x_min + x_max) / 2, (y_min + y_max) / 2
        w, h = x_max - x_min, y_max - y_min
        result.append(Detection(points=np.array([xc, yc]), scores=np.array([score]), data=np.array([w, h, frame_id])))
        
    return result

def euclidean_distance(detection, tracked_object):
    return np.linalg.norm(detection.points - tracked_object.estimate)

def tracking_function(tracker, frame_id, bboxes, scores, best_conf, num_prev_bbox=None):
    
    detects = []
    predictions = []
    
    if len(scores)>0:
        for i in range(len(bboxes)):
            # remember to check
            if scores[i] <= best_conf:
                continue
            box = bboxes[i]
            score = scores[i]
            x_min = int(box[0])
            y_min = int(box[1])
            bbox_width = int(box[2])
            bbox_height = int(box[3])
            detects.append([x_min, y_min, x_min+bbox_width, y_min+bbox_height, score])
            predictions.append('{:.2f} {} {} {} {}'.format(score, x_min, y_min, bbox_width, bbox_height))
#             print(predictions[:-1])
    # Update tracks using detects from current frame
    tracked_objects = tracker.update(detections=to_norfair(detects, frame_id))
    to_add_preds = []
    for tobj in tracked_objects:
        bbox_width, bbox_height, last_detected_frame_id = tobj.last_detection.data
        if last_detected_frame_id == frame_id:  # Skip objects that were detected on current frame
            continue
        xc, yc = tobj.estimate[0]
        x_min, y_min = int(round(xc - bbox_width / 2)), int(round(yc - bbox_height / 2))

        #exclude those in the edge
        if (x_min + bbox_width >= 1279) or (y_min + bbox_height) >= 719 or (x_min <= 1) or (y_min <= 1):
            continue
        score = tobj.last_detection.scores[0]            
        to_add_preds.append('{:.2f} {} {} {} {}'.format(score, x_min, y_min, bbox_width, bbox_height))
        #predictions.append('{:.2f} {} {} {} {}'.format(score, x_min, y_min, bbox_width, bbox_height))
            
        
    if (num_prev_bbox is None or (num_prev_bbox - len(predictions) + 1) >= len(to_add_preds)):
        predictions.extend(to_add_preds)
        
    return predictions

In [24]:
df_perf_track_dict = {model_version: pd.DataFrame(columns=["all"] + [str(round(th,3)) for th in np.arange(0.3, 0.85, 0.05)]).copy() for model_version in model_versions}
df_oof.sort_values(["video_id","video_frame"], inplace=True)
df_oof["track_pred"] = [[] for i in range(len(df_oof))]
f2_scores_track = {}
P_scores_track = {}
R_scores_track = {}
f2_specific_scores_track = {k:{} for k in f2_specific_keys}

for model_version in model_versions:
    BEST_CONF = best_conf_dict[model_version]
    pred_col = "pred_" + model_version + f"_fold1"
    prd_bboxs_tracking_list = []
    frame_id = 0
    tracker = Tracker(
        distance_function=euclidean_distance, 
        distance_threshold=30,
        hit_inertia_min=3,
        hit_inertia_max=6,
        initialization_delay=2,
    )          
    num_prev_bbox = None
    for idx in df_oof.index:
        row = df_oof.loc[idx]
        prd_bboxs = row[pred_col]
        bboxes, confis = [p[1:] for p in prd_bboxs], [p[0] for p in prd_bboxs]    
        predictions = tracking_function(tracker, frame_id, bboxes, confis, BEST_CONF, num_prev_bbox)    
        prd_bboxs_tracking =  [[float(p) for p in pred.split(" ")] for pred in predictions]
        num_prev_bbox = len(bboxes)
        prd_bboxs_tracking_list.append(np.array(prd_bboxs_tracking))
        df_oof.at[idx, "track_pred"].extend(prd_bboxs_tracking)
        frame_id += 1        
        
    f2_dict = util.calc_f2_score(gt_bboxs_list, prd_bboxs_tracking_list, verbose=False) 
    f2_scores_track[model_version] = f2_dict['f2']
    P_scores_track[model_version] = f2_dict['P']
    R_scores_track[model_version] = f2_dict['R']
    for k in f2_specific_keys:
        f2_specific_scores_track[k][model_version] = f2_dict[k]
    
    
    df_perf_track_dict[model_version].loc["f2",:] = [f2_dict[col] for col in f_cols]
    df_perf_track_dict[model_version].loc["P",:] = [f2_dict[col] for col in P_cols]
    df_perf_track_dict[model_version].loc["R",:] = [f2_dict[col] for col in R_cols]
    df_oof[f'{model_version}_tps'] = f2_dict["tps"]
    df_oof[f'{model_version}_fps'] = f2_dict["fps"]
    df_oof[f'{model_version}_fns'] = f2_dict["fns"]
    print("Using Tracking new: model {}, conf_th {:.2f}, f2 {:.3f}, precision {:.3f}, recall {:.3f}".format(model_version, BEST_CONF, f2_dict['f2'], f2_dict['P'], f2_dict['R']))
    display(df_perf_track_dict[model_version].astype(float).style.format("{:.3f}").background_gradient(axis=1))

Using Tracking new: model best_s6_3600_GT, conf_th 0.30, f2 0.714, precision 0.614, recall 0.744


,all,0.3,0.35,0.4,0.45,0.5,0.55,0.6,0.65,0.7,0.75,0.8
f2,0.714,0.841,0.840,0.837,0.833,0.826,0.806,0.773,0.720,0.607,0.475,0.296
P,0.614,0.724,0.723,0.720,0.717,0.710,0.694,0.665,0.620,0.522,0.409,0.255
R,0.744,0.877,0.875,0.872,0.868,0.861,0.840,0.806,0.751,0.633,0.495,0.309


Using Tracking new: model 0209_m6_B_LS02_newLGBT, conf_th 0.10, f2 0.708, precision 0.627, recall 0.732


,all,0.3,0.35,0.4,0.45,0.5,0.55,0.6,0.65,0.7,0.75,0.8
f2,0.708,0.828,0.827,0.826,0.824,0.816,0.800,0.771,0.714,0.607,0.469,0.309
P,0.627,0.734,0.733,0.732,0.730,0.723,0.709,0.683,0.632,0.538,0.415,0.274
R,0.732,0.856,0.854,0.853,0.851,0.843,0.827,0.796,0.737,0.627,0.484,0.319


Using Tracking new: model 0207_m6_B_LS02_newGT_clahe, conf_th 0.25, f2 0.702, precision 0.648, recall 0.717


...

,all,0.3,0.35,0.4,0.45,0.5,0.55,0.6,0.65,0.7,0.75,0.8
f2,0.717,0.842,0.841,0.841,0.836,0.826,0.806,0.774,0.717,0.616,0.478,0.315
P,0.643,0.754,0.754,0.754,0.749,0.740,0.722,0.693,0.642,0.552,0.428,0.282
R,0.739,0.867,0.866,0.866,0.861,0.851,0.830,0.797,0.738,0.634,0.492,0.325


Using Tracking new: model 0209_s6_B_LS02_newGT_img4096_valid, conf_th 0.35, f2 0.709, precision 0.709, recall 0.709


,all,0.3,0.35,0.4,0.45,0.5,0.55,0.6,0.65,0.7,0.75,0.8
f2,0.709,0.834,0.832,0.831,0.827,0.822,0.804,0.767,0.711,0.603,0.473,0.293
P,0.709,0.834,0.832,0.831,0.827,0.822,0.804,0.767,0.711,0.603,0.473,0.293
R,0.709,0.834,0.832,0.831,0.827,0.822,0.804,0.767,0.711,0.603,0.473,0.293


Using Tracking new: model GT3_4680, conf_th 0.33, f2 0.721, precision 0.630, recall 0.749


,all,0.3,0.35,0.4,0.45,0.5,0.55,0.6,0.65,0.7,0.75,0.8
f2,0.721,0.847,0.846,0.845,0.842,0.831,0.815,0.790,0.723,0.626,0.477,0.294
P,0.630,0.739,0.738,0.737,0.735,0.725,0.711,0.689,0.631,0.546,0.416,0.257
R,0.749,0.879,0.878,0.877,0.874,0.863,0.846,0.820,0.750,0.649,0.495,0.305


In [25]:
f2_score_t_df = pd.DataFrame.from_dict(f2_scores_track,orient='index',columns=["f2_t"])
P_score_t_df = pd.DataFrame.from_dict(P_scores_track,orient='index',columns=["P_t"])
R_score_t_df = pd.DataFrame.from_dict(R_scores_track,orient='index',columns=["R_t"])
f2_s_t_df = []
for k in f2_specific_keys:
    f2_s_t_df.append(pd.DataFrame.from_dict(f2_specific_scores_track[k],orient='index',columns=[k]))
score_df_t = pd.concat([f2_score_t_df, P_score_t_df, R_score_t_df] + f2_s_t_df, axis=1)
display(score_df.style.format("{:.3f}").background_gradient(axis=0))
display(score_df_t.style.format("{:.3f}").background_gradient(axis=0))

,f2,P,R,f2_0.3,f2_0.35,f2_0.4,f2_0.45,f2_0.5,f2_0.55,f2_0.6,f2_0.65,f2_0.7,f2_0.75,f2_0.8
best_s6_3600_GT,0.702,0.636,0.720,0.817,0.817,0.816,0.813,0.808,0.794,0.762,0.712,0.605,0.475,0.298
0209_m6_B_LS02_newLGBT,0.691,0.648,0.702,0.794,0.794,0.793,0.793,0.791,0.780,0.756,0.705,0.604,0.473,0.313
0207_m6_B_LS02_newGT_clahe,0.695,0.667,0.703,0.801,0.801,0.799,0.798,0.795,0.785,0.759,0.709,0.611,0.472,0.319
0208_s6_B_LS02_newGT,0.666,0.636,0.674,0.772,0.772,0.772,0.772,0.769,0.753,0.721,0.667,0.587,0.452,0.292
0208_m_TPH_1900_B_LS02_newGT,0.640,0.621,0.644,0.736,0.736,0.736,0.736,0.735,0.729,0.704,0.646,0.559,0.438,0.282
0209_m6_B_LS02_LGBT_newP,0.703,0.683,0.708,0.806,0.806,0.806,0.806,0.802,0.789,0.764,0.710,0.631,0.493,0.320
r27-0210-yolov5m6-2400,0.704,0.671,0.713,0.815,0.815,0.815,0.812,0.804,0.790,0.764,0.711,0.617,0.481,0.318
0209_s6_B_LS02_newGT_img4096_valid,0.692,0.732,0.682,0.804,0.803,0.803,0.801,0.797,0.784,0.753,0.700,0.598,0.472,0.293
GT3_4680,0.717,0.662,0.732,0.824,0.824,0.824,0.824,0.821,0.810,0.782,0.733,0.638,0.486,0.319


,f2_t,P_t,R_t,f2_0.3,f2_0.35,f2_0.4,f2_0.45,f2_0.5,f2_0.55,f2_0.6,f2_0.65,f2_0.7,f2_0.75,f2_0.8
best_s6_3600_GT,0.714,0.614,0.744,0.841,0.840,0.837,0.833,0.826,0.806,0.773,0.720,0.607,0.475,0.296
0209_m6_B_LS02_newLGBT,0.708,0.627,0.732,0.828,0.827,0.826,0.824,0.816,0.800,0.771,0.714,0.607,0.469,0.309
0207_m6_B_LS02_newGT_clahe,0.702,0.648,0.717,0.817,0.816,0.812,0.811,0.803,0.793,0.764,0.709,0.610,0.469,0.316
0208_s6_B_LS02_newGT,0.684,0.610,0.705,0.806,0.804,0.801,0.799,0.793,0.774,0.737,0.677,0.590,0.450,0.289
0208_m_TPH_1900_B_LS02_newGT,0.655,0.592,0.673,0.766,0.764,0.763,0.759,0.756,0.748,0.718,0.653,0.560,0.434,0.280
0209_m6_B_LS02_LGBT_newP,0.714,0.659,0.730,0.831,0.829,0.829,0.824,0.817,0.799,0.771,0.713,0.632,0.492,0.318
r27-0210-yolov5m6-2400,0.717,0.643,0.739,0.842,0.841,0.841,0.836,0.826,0.806,0.774,0.717,0.616,0.478,0.315
0209_s6_B_LS02_newGT_img4096_valid,0.709,0.709,0.709,0.834,0.832,0.831,0.827,0.822,0.804,0.767,0.711,0.603,0.473,0.293
GT3_4680,0.721,0.630,0.749,0.847,0.846,0.845,0.842,0.831,0.815,0.790,0.723,0.626,0.477,0.294


# Ensemble

In [26]:
sizes = np.concatenate(df['real_bbox'].apply(lambda x: [p[2]*p[3] for p in x]).values)
np.quantile(sizes, 0.33) / (1280 * 720)

0.0014322916666666666

In [27]:
from image_similarity_measures.quality_metrics import rmse
from ensemble_boxes import weighted_boxes_fusion, nms, non_maximum_weighted, soft_nms

def modify_size(boxes, to01=True):
    if len(boxes) > 0:
        if to01:
            boxes[:,2:] = boxes[:,2:] + boxes[:,:2]
            boxes[:,0] = boxes[:,0] / 1279.
            boxes[:,1] = boxes[:,1] / 719.
            boxes[:,2] = boxes[:,2] / 1279.
            boxes[:,3] = boxes[:,3] / 719.
        else:
            boxes[:,0] = boxes[:,0] * 1279.
            boxes[:,1] = boxes[:,1] * 719.
            boxes[:,2] = boxes[:,2] * 1279.
            boxes[:,3] = boxes[:,3] * 719.
            boxes[:,2:] = boxes[:,2:] - boxes[:,:2]
        
    return boxes
    
def run_wbf(bboxes, confs, iou_thr=0.5, skip_box_thr=0.1, keep_small=False, small_thresh=0.001, verbose=False):
    if len(bboxes) == 1:
        return bboxes[0], confs[0], []   
    else:
        for i in range(len(bboxes)):
            bboxes[i] = modify_size(bboxes[i], to01=True)    
    if keep_small:
        # for samll objects, we use NMS
        small_bboxes_num = 0
        small_bboxes = []
        small_confs = []
        for i in range(len(bboxes)):
            if len(bboxes[i]) != 0:
                sub_bboxes = bboxes[i].copy()
                sub_confs = confs[i].copy()
                sizes = (sub_bboxes[:,3] - sub_bboxes[:, 1]) * (sub_bboxes[:,2] - sub_bboxes[:, 0])
                small_place = np.where(sizes <= small_thresh)[0]
                
                sub_bboxes = sub_bboxes[small_place]
                sub_confs = np.array(sub_confs)[small_place].tolist()
                bboxes[i] = np.delete(bboxes[i], small_place, axis=0)
                confs[i] = np.delete(np.array(confs[i]), small_place, axis=0).tolist()
            else:
                sub_bboxes = np.empty((0,4))
                sub_confs = []
                
            small_bboxes.append(sub_bboxes)
            small_confs.append(sub_confs)            
            small_bboxes_num += len(sub_bboxes)
        small_labels = [np.ones(len(conf)) for conf in small_confs]
        if small_bboxes_num > 0:
            small_boxes, small_scores, small_labels = nms(small_bboxes, small_confs, small_labels, iou_thr=0.7)
        else:
            small_boxes, small_scores, small_labels = np.empty((0,4)), [], []
        if verbose:
            if len(small_boxes) > 0:
                print("Small bboxes number {}, after NMS {}".format(small_bboxes_num, len(small_boxes) ))
        
    labels = [np.ones(len(conf)) for conf in confs]
    boxes, scores, labels = weighted_boxes_fusion(bboxes, confs, labels, iou_thr=iou_thr, skip_box_thr=0.001, allows_overflow=True, conf_type='avg')
    
    boxes = modify_size(boxes, to01=False)  
    boxes = [box for i,box in enumerate(boxes) if scores[i] >= skip_box_thr]
    scores = [conf for conf in scores if conf >= skip_box_thr]

    if keep_small:
        small_boxes = modify_size(small_boxes, to01=False)  
        boxes = boxes + [b for b in small_boxes]
        scores = scores + [s for s in small_scores]

    return boxes, scores, labels

In [28]:
f2_score_df.index

Index(['best_s6_3600_GT', '0209_m6_B_LS02_newLGBT',
       '0207_m6_B_LS02_newGT_clahe', '0208_s6_B_LS02_newGT',
       '0208_m_TPH_1900_B_LS02_newGT', '0209_m6_B_LS02_LGBT_newP',
       'r27-0210-yolov5m6-2400', '0209_s6_B_LS02_newGT_img4096_valid',
       'GT3_4680'],
      dtype='object')

In [29]:
df_oof["wbf"] = None
df_oof['notrack_pred'] = None
df_oof['track_pred'] = None
write_track = True

keep_small = False
small_thresh = 0.001

iou_thr = 0.4

wbf_models =['r27-0210-yolov5m6-2400', '0209_s6_B_LS02_newGT_img4096_valid', 'GT3_4680', '0207_m6_B_LS02_newGT_clahe']
df_perf_track_dict = {model_version: pd.DataFrame(columns=["all"] + [str(round(th,3)) for th in np.arange(0.3, 0.85, 0.05)]).copy() for model_version in ['wbf']}
df_perf_notrack_dict = {model_version: pd.DataFrame(columns=["all"] + [str(round(th,3)) for th in np.arange(0.3, 0.85, 0.05)]).copy() for model_version in ['wbf']}

num_prev_bbox = None
for skip_box_thr in np.arange(1.6, 1.7, 0.2):
    skip_box_thr = skip_box_thr / 10
    prd_wbf_list = []
    prd_wbf_list_notrack = []
    frame_id = 0
    tracker = Tracker(
        distance_function=euclidean_distance, 
        distance_threshold=30,
        hit_inertia_min=3,
        hit_inertia_max=6,
        initialization_delay=2,
    ) 
    prev_sim = None
    prev_image = None
    for i, idx in enumerate(df_oof.index):
        row = df_oof.loc[idx]
        bboxes_l = []
        confs_l = []                                    
        for model_version in wbf_models:
            BEST_CONF = best_conf_dict[model_version]
            pred_col = "pred_" + model_version + f"_fold1"
            prd_bboxs = row[pred_col]
            prd_bboxs = [p for p in prd_bboxs if (p[0] >= BEST_CONF and p[3] * p[4] <= 15000)]
            bboxes, confis = [p[1:] for p in prd_bboxs], [p[0] for p in prd_bboxs]    
            bboxes = np.array(bboxes,dtype=np.float64)
            bboxes_l.append(bboxes)
            confs_l.append(confis)

        wbf_boxes, wbf_confs, _ = run_wbf(bboxes_l, confs_l, iou_thr, skip_box_thr, keep_small, small_thresh, verbose=False)
        predictions = tracking_function(tracker, frame_id, wbf_boxes, wbf_confs, 0, num_prev_bbox)
        prd_bboxs_tracking =  [[float(p) for p in pred.split(" ")] for pred in predictions]
        num_prev_bbox = len(wbf_boxes)
        #num_prev_bbox = len(prd_bboxs_tracking)
        if write_track:
            df_oof.at[idx, "notrack_pred"] = [[wbf_confs[i]] + wbf_boxes[i].tolist() for i in range(len(wbf_boxes))]
            df_oof.at[idx, "track_pred"] = prd_bboxs_tracking
        prd_wbf_list.append(np.array(prd_bboxs_tracking))
        prd_wbf_list_notrack.append(np.array([np.array([wbf_confs[i]] + wbf_boxes[i].tolist()) for i in range(len(wbf_boxes))]))


        frame_id += 1

    f2_dict = util.calc_f2_score(gt_bboxs_list, prd_wbf_list, verbose=False) 
    f2_dict_notrack =  util.calc_f2_score(gt_bboxs_list, prd_wbf_list_notrack, verbose=False) 
    print('Skip_box_thr {:.2f}, models {}'.format(skip_box_thr, wbf_models))
    print("Tracking f2 {:.3f}, precision {:.3f}, recall {:.3f}".format(f2_dict['f2'], f2_dict['P'], f2_dict['R']))
    print("No Track f2 {:.3f}, precision {:.3f}, recall {:.3f}".format(f2_dict_notrack['f2'], f2_dict_notrack['P'], f2_dict_notrack['R']))
    
    if write_track:
        df_oof[f'wbf_tps'] = f2_dict["tps"]
        df_oof[f'wbf_fps'] = f2_dict["fps"]
        df_oof[f'wbf_fns'] = f2_dict["fns"]
        df_perf_track_dict['wbf'].loc["f2",:] = [f2_dict[col] for col in f_cols]
        df_perf_track_dict['wbf'].loc["P",:] = [f2_dict[col] for col in P_cols]
        df_perf_track_dict['wbf'].loc["R",:] = [f2_dict[col] for col in R_cols]        
        display(df_perf_track_dict['wbf'].astype(float).style.format("{:.3f}").background_gradient(axis=1).set_caption("tracking"))
        
        df_perf_notrack_dict['wbf'].loc["f2",:] = [f2_dict_notrack[col] for col in f_cols]
        df_perf_notrack_dict['wbf'].loc["P",:] = [f2_dict_notrack[col] for col in P_cols]
        df_perf_notrack_dict['wbf'].loc["R",:] = [f2_dict_notrack[col] for col in R_cols]        
        display(df_perf_notrack_dict['wbf'].astype(float).style.format("{:.3f}").background_gradient(axis=1).set_caption("no tracking"))

/home/vincent/.local/lib/python3.8/site-packages/ensemble_boxes/ensemble_boxes_wbf.py:64: UserWarning: X1 < 0 in box. Set it to 0.
  warnings.warn('X1 < 0 in box. Set it to 0.')
/home/vincent/.local/lib/python3.8/site-packages/ensemble_boxes/ensemble_boxes_wbf.py:73: UserWarning: X2 > 1 in box. Set it to 1. Check that you normalize boxes in [0, 1] range.
  warnings.warn('X2 > 1 in box. Set it to 1. Check that you normalize boxes in [0, 1] range.')
/home/vincent/.local/lib/python3.8/site-packages/ensemble_boxes/ensemble_boxes_wbf.py:85: UserWarning: Y2 > 1 in box. Set it to 1. Check that you normalize boxes in [0, 1] range.
  warnings.warn('Y2 > 1 in box. Set it to 1. Check that you normalize boxes in [0, 1] range.')
/home/vincent/.local/lib/python3.8/site-packages/ensemble_boxes/ensemble_boxes_wbf.py:76: UserWarning: Y1 < 0 in box. Set it to 0.
  warnings.warn('Y1 < 0 in box. Set it to 0.')


Skip_box_thr 0.16, models ['r27-0210-yolov5m6-2400', '0209_s6_B_LS02_newGT_img4096_valid', 'GT3_4680', '0207_m6_B_LS02_newGT_clahe']
Tracking f2 0.757, precision 0.715, recall 0.768
No Track f2 0.756, precision 0.746, recall 0.758


,all,0.3,0.35,0.4,0.45,0.5,0.55,0.6,0.65,0.7,0.75,0.8
f2,0.757,0.878,0.878,0.877,0.876,0.868,0.851,0.824,0.767,0.668,0.505,0.332
P,0.715,0.829,0.829,0.828,0.827,0.820,0.804,0.778,0.725,0.631,0.477,0.313
R,0.768,0.891,0.891,0.890,0.889,0.881,0.864,0.836,0.779,0.678,0.512,0.337


,all,0.3,0.35,0.4,0.45,0.5,0.55,0.6,0.65,0.7,0.75,0.8
f2,0.756,0.854,0.854,0.854,0.854,0.852,0.842,0.819,0.775,0.688,0.544,0.376
P,0.746,0.844,0.844,0.844,0.844,0.841,0.831,0.809,0.766,0.680,0.537,0.371
R,0.758,0.857,0.857,0.857,0.857,0.854,0.844,0.821,0.778,0.690,0.546,0.377


In [ ]:
# Skip_box_thr 0.16, models ['r27-0210-yolov5m6-2400', '0209_s6_B_LS02_newGT_img4096_valid', 'GT3_4680', '0207_m6_B_LS02_newGT_clahe']
# Tracking f2 0.757, precision 0.715, recall 0.768
# No Track f2 0.756, precision 0.746, recall 0.758

# Analyze Prediction

In [ ]:
sizes = {}
confs = {}
for model_v in wbf_models + ['track_pred']:
    sizes[model_v] = []
    confs[model_v] = []
    
for idx,row in tqdm(df_oof.iterrows()):
    for model_v in wbf_models + ['track_pred']:
        if model_v in wbf_models:
            p = row["pred_" + model_v + "_fold1"]
            
            p = [x for x in p if (x[0] >= best_conf_dict[model_v] and x[3] * x[4] <= 15000)]
        else:
            p = row[model_v]
        if len(p) >= 1:
            sizes[model_v].extend([x[3] * x[4] for x in p])
            confs[model_v].extend([x[0] for x in p])            

In [ ]:
gt_sizes = []
for idx,row in df_oof.iterrows():
    g = row['real_bbox']
    if len(g) >= 1:
        gt_sizes.extend([x[2] * x[3] for x in g])    

In [ ]:
import seaborn as sns
plt.subplots(1,1,figsize=(9,9))
sns.histplot(gt_sizes, bins=100)
plt.show()

In [ ]:
fig, axes = plt.subplots(1,2,figsize=(24,12))

for model_v in wbf_models + ['track_pred']:
    sns.scatterplot(x=sizes[model_v], y=confs[model_v], label=model_v, alpha=0.5, ax=axes[0])
    
axes[0].legend()
axes[0].set_xlabel("pred size", size=12)
axes[0].set_ylabel("conf", size=12)

for model_v in wbf_models + ['track_pred']:
    sns.scatterplot(x=sizes[model_v], y=confs[model_v], label=model_v, alpha=0.5, ax=axes[1])
    
axes[1].legend()
axes[1].set_xlabel("pred size", size=12)
axes[1].set_ylabel("conf", size=12)
axes[1].set_xlim(0, 12000)
plt.show()

In [ ]:
# basic_cols = ['video_id', 'sequence', 'video_frame', 'sequence_frame', 'image_id',
#        'annotations', 'fold', 'old_image_path', 'image_path', 'label_path',
#        'real_bbox', 'num_bbox']
# basic_cols = basic_cols + ['notrack_pred']
# util.save_pickle(df_oof[basic_cols],"../input/wbf_notrack_pred.pkl")

# check image

In [ ]:
check_versions = ["wbf"]
f2_score_df.loc[check_versions[:-1]].style.format("{:.3f}").background_gradient(axis=0)

In [ ]:
df_oof['tp'] = 0
df_oof['fp'] = 0
df_oof['fn'] = 0
for key in check_versions: 
    df_oof[f'{key}_tp'] = df_oof[f'{key}_tps'].apply(np.sum)
    df_oof[f'{key}_fp'] = df_oof[f'{key}_fps'].apply(np.sum)
    df_oof[f'{key}_fn'] = df_oof[f'{key}_fns'].apply(np.sum)
    df_oof['tp'] = df_oof['tp'] + df_oof[f'{key}_tp']
    df_oof['fp'] = df_oof['fp'] + df_oof[f'{key}_fp']
    df_oof['fn'] = df_oof['fn'] + df_oof[f'{key}_fn']

In [ ]:
color_pred = [(0,125,0)]    
color_real = [(255,0,0)]
def plot_img(df, idx, keys, save=False, pre=""):
    row = df.iloc[idx]
    path = row["image_path"]
    img = cv2.imread(str(path))[...,::-1]
    real_bboxes = row["real_bbox"]
    preds = row["track_pred"]
#     for key in keys:
#         best_conf = best_conf_dict[key]
#         pred_col = "pred_" + key + "_fold1"
#         if len(row[pred_col]) > 0:
#             preds = preds + [p for p in row[pred_col] if p[0] >= best_conf]
            
    bboxes = [p[1:5] for p in preds]
    confis = [p[0] for p in preds]
    print(f"video {row.video_id}, sequence {row.sequence}, frame {row.video_frame}")
    img_plot = util.show_img(img, 
                          bboxes + real_bboxes, 
                          confis + [1] * len(real_bboxes),
                          color_pred  * len(bboxes) + color_real * len(real_bboxes), 
                          bbox_format='coco')
    if save:
        img_plot.save(f"./pic_check/{pre}v{row.video_id}_frame{row.video_frame}_seq{row.sequence}.jpg")
    display(img_plot)

In [ ]:
for i, iou in tqdm(enumerate(np.arange(0.3,0.85,0.05))):
    df_oof[f"f2_{iou:.2f}"] = 0
    df_oof[f"P_{iou:.2f}"] = 0
    df_oof[f"R_{iou:.2f}"] = 0
    for idx in df_oof.index:
        tp = 0
        fp = 0
        fn = 0
        for key in check_versions:
            tp += df_oof.loc[idx, f'{key}_tps'][i]
            fp += df_oof.loc[idx, f'{key}_fps'][i]
            fn += df_oof.loc[idx, f'{key}_fns'][i]
        df_oof.loc[idx, f"f2_{iou:.2f}"] = util.f_beta(tp, fp, fn, beta=2)
        df_oof.loc[idx, f"P_{iou:.2f}"] = util.f_beta(tp, fp, fn, beta=0)
        df_oof.loc[idx, f"R_{iou:.2f}"] = util.f_beta(tp, fp, fn, beta=100)

In [ ]:
df_oof["f2"] = df_oof[[f'f2_{iou:.2f}' for iou in np.arange(0.3,0.85,0.5)]].mean(axis=1)
df_oof["P"] = df_oof[[f'P_{iou:.2f}' for iou in np.arange(0.3,0.85,0.5)]].mean(axis=1)
df_oof["R"] = df_oof[[f'R_{iou:.2f}' for iou in np.arange(0.3,0.85,0.5)]].mean(axis=1)
#df_oof.head()

In [ ]:
show_col = ["video_id","sequence","video_frame","num_bbox", "f2", "P", "R"]
sub_col = ["fp"] + [key+"_fp" for key in check_versions]
sub_col = sub_col + ["tp"] + [key+"_tp" for key in check_versions]
sub_col = sub_col + ["fn"] + [key+"_fn" for key in check_versions]

## most FP among empy group

In [ ]:
df_neg = df_oof.query("num_bbox == 0").copy()
df_neg.sort_values("fp", ascending=False, inplace=True)
df_neg.reset_index(inplace=True, drop=True)
df_neg[show_col + sub_col].head()

In [ ]:
for i in range(5):
    plot_img(df_neg, i, model_versions, save=False)

## Most FP among non-empty group

In [ ]:
df_pos = df_oof.query("num_bbox > 0").copy()
df_pos.sort_values("fp", ascending=False, inplace=True)
df_pos.reset_index(inplace=True, drop=True)
df_pos[show_col + sub_col].head()

In [ ]:
for i in range(5):
    plot_img(df_pos, i, model_versions)

## Most TP

In [ ]:
df_pos = df_oof.query("num_bbox > 0").copy()
df_pos.sort_values("tp", ascending=False, inplace=True)
df_pos.reset_index(inplace=True, drop=True)
df_pos[show_col + sub_col].head(5)

In [ ]:
for i in range(3):
    plot_img(df_pos, i, model_versions)

## Most FN

In [ ]:
df_pos = df_oof.query("num_bbox > 0").copy()
df_pos.sort_values("fn", ascending=False, inplace=True)
df_pos.reset_index(inplace=True, drop=True)
df_pos[show_col + sub_col].head(10)

In [ ]:
for i in range(10):
    plot_img(df_pos, i, model_versions)

# Make Video

In [ ]:
df_oof.groupby(["video_id","sequence"]).size()

In [ ]:
from IPython.display import Image
from PIL import Image as Img
import subprocess
#This code I found in: https://www.kaggle.com/bamps53/create-annotated-video Thank you for sharing.
out_dir = "/home/vincent/Kaggle/data/tensorflow-great-barrier-reef/video_check/"
if not os.path.exists(out_dir):
    os.makedirs(out_dir)

def load_image(image_dir):
    assert os.path.exists(image_dir), f'{image_dir} does not exist.'
    img = cv2.imread(str(image_dir))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img

def load_image_with_annotations(image_dir, annotations):
    img = load_image(image_dir)
    if len(annotations) > 0:
        for ann in annotations:
            cv2.rectangle(img, (ann['x'], ann['y']),
                (ann['x'] + ann['width'], ann['y'] + ann['height']),
                (0, 255, 0), thickness=2,)
    return img

def show_prediction(img, bboxes, scores, show = True):
    colors = [(0, 0, 255)]

    obj_names = ["s"]

    for box, score in zip(bboxes, scores):
        cv2.rectangle(img, (int(box[0]), int(box[1])), (int(box[0] + box[2]), int(box[1] + box[3])), (255,0,0), 2)
        cv2.putText(img, f'{score:.2f}', (int(box[0]), int(box[1])-3), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,0,0), 2, cv2.LINE_AA)
    if show:
        img = Img.fromarray(img).resize((1280, 720))
    return img

def make_video(df, video_id, sequence_id, pred_col, out_dir, best_conf, name=None, s_f=None, e_f=None):
    fps = 15
    width = 1280
    height = 720
    if name is None:
        name = "video"
    
    save_path = f'{out_dir}/{name}.mp4'
    tmp_path =  f'{out_dir}/tmp-{name}.mp4'
    output_video = cv2.VideoWriter(tmp_path, cv2.VideoWriter_fourcc(*"MP4V"), fps, (width, height),)
    
    # I just generate ony part of video
    if video_id is not None and sequence_id is not None:
        query = 'video_id == {} and sequence == {}'.format(video_id, sequence_id)
        if s_f is not None:
            query = query + 'and video_frame >= {} and video_frame <= {}'.format(s_f, e_f)
        print(query)
        video_df = df.query(query)
    else:
        video_df = df
    print(video_df.shape)
    for _, row in tqdm(video_df.iterrows(), total=len(video_df)):
        video_id = row.video_id
        video_frame = row.video_frame
        annotations = row.annotations
        img_file = row.image_path
        img = load_image_with_annotations(img_file, annotations)
        preds = row[pred_col]
        best_conf = 0
        bboxes = [preds[i][1:] for i in range(len(preds)) if preds[i][0] >= best_conf]
        confis = [preds[i][0] for i in range(len(preds)) if preds[i][0] >= best_conf]        
        img = show_prediction(img, bboxes, confis, show=False)
        cv2.putText(img, f'{video_id}-{video_frame}', (10,70), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0), 1, cv2.LINE_AA)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        output_video.write(img)
    
    output_video.release()

    if os.path.exists(save_path):
        os.remove(save_path)
    subprocess.run(
        ["ffmpeg", "-i", tmp_path, "-crf", "18", "-preset", "veryfast", "-vcodec", "libx264", save_path]
    )
    os.remove(tmp_path)

In [ ]:
for video_id, sequence in df_oof.groupby(["video_id","sequence"]).size().index:
    name = f'wbf-{video_id}-{sequence}'
    make_video(df_oof, video_id, sequence, 'track_pred', out_dir, 0, name=name)

In [ ]:
from base64 import b64encode
def play(filename):
    html = ''
    video = open(filename,'rb').read()
    src = 'data:video/mp4;base64,' + b64encode(video).decode()
    html += '<video width=1280 controls autoplay loop><source src="%s" type="video/mp4"></video>' % src 
    return HTML(html)

In [ ]:
video_id = 1
sequence = 15827
s_f = 2148
e_f = 2205
q = f"video_id == {video_id} and sequence == {sequence} and video_frame >= {s_f} and video_frame <= {e_f}"
df_oof_check = df_oof.reset_index(drop=True).query(q).copy()

In [ ]:
df_oof_check.index

In [ ]:
f2_dict = util.calc_f2_score(gt_bboxs_list, prd_wbf_list, verbose=False) 
f2_dict['f2']

In [ ]:
gt_bboxs_list_modify = gt_bboxs_list.copy()
prd_wbf_list_modify = prd_wbf_list.copy()

In [ ]:
# for idx in df_oof_check.index:
#     prd_wbf_list_modify[idx] = np.array([])
for idx in df_oof_check.index:
    pred = prd_wbf_list_modify[idx]
    if len(pred) > 0:
        gt_bboxs_list_modify[idx] = pred[:,1:]

In [ ]:
# for idx in df_oof_check.index:
#     print(gt_bboxs_list_modify[idx])

In [ ]:
prd_wbf_list_modify[idx]

In [ ]:
f2_dict = util.calc_f2_score(gt_bboxs_list_modify, prd_wbf_list_modify, verbose=False) 
f2_dict['f2']

In [ ]:
(df_oof_check['track_pred'].apply(len) > 0).sum()

In [ ]:
video_id = 2
sequence =26651
s_f =4154
e_f =4200

name = f'UnkownFP_wbf-{video_id}-{sequence}-{s_f}-{e_f}'
make_video(df_oof, video_id, sequence, 'track_pred', out_dir, 0, name=name,s_f=s_f, e_f=e_f)

In [ ]:
df_oof['track_pred']

In [ ]:
video_id = 2
sequence =26651
s_f =4639
e_f =4748

name = f'FP_wbf-{video_id}-{sequence}-{s_f}-{e_f}'
make_video(df_oof, video_id, sequence, 'track_pred', out_dir, 0, name=name,s_f=s_f, e_f=e_f)

In [ ]:
video_id = 2
sequence =26651
s_f =5024
e_f =5090

name = f'Fish_wbf-{video_id}-{sequence}-{s_f}-{e_f}'
make_video(df_oof, video_id, sequence, 'track_pred', out_dir, 0, name=name,s_f=s_f, e_f=e_f)

In [ ]:
video_id = 2
sequence =26651
s_f =5120
e_f =5160

name = f'Fish_wbf-{video_id}-{sequence}-{s_f}-{e_f}'
make_video(df_oof, video_id, sequence, 'track_pred', out_dir, 0, name=name,s_f=s_f, e_f=e_f)

# Improve FN

In [ ]:
egs = [
    # sequence , frame id
    (53708, 8926),
    (53708, 8928),
    (53708, 9654),
    (53708, 9655),
]

In [ ]:

def plot_img_2(df, sequence, frame_id, keys, save=False, pre="", use_clahe=False, aug=None):
    #row = df.iloc[idx]
    row = df.query("sequence == @seq and video_frame == @frame_id").iloc[0]
    path = row["image_path"]
    img = cv2.imread(str(path))[...,::-1]
    if use_clahe:
        for i in range(3):
            img[:, :, i] = clahe.apply((img[:, :, i]))         
    if aug is not None:
        img = aug(image=img)['image']
    real_bboxes = row["real_bbox"]
    preds = row["track_pred"]            
    bboxes = [p[1:5] for p in preds]
    confis = [p[0] for p in preds]
    print(f"video {row.video_id}, sequence {row.sequence}, frame {row.video_frame}")
    img_plot = util.show_img(img, 
                          bboxes + real_bboxes, 
                          confis + [1] * len(real_bboxes),
                          color_pred  * len(bboxes) + color_real * len(real_bboxes), 
                          bbox_format='coco')
    if save:
        img_plot.save(f"./pic_check/{pre}v{row.video_id}_frame{row.video_frame}_seq{row.sequence}.jpg")
    display(img_plot)

In [ ]:
seq, frame = egs[0]
seq_tp = 53708
frame_tp = 9653
plot_img_2(df_oof, seq, frame, model_versions, use_clahe=False)
plot_img_2(df_oof, seq_tp, frame_tp, model_versions, use_clahe=False)

## CLAHE

In [ ]:
clahe = cv2.createCLAHE(clipLimit=5, tileGridSize=(10, 10))
plot_img_2(df_oof, seq, frame, model_versions, use_clahe=True)
plot_img_2(df_oof, seq_tp, frame_tp, model_versions, use_clahe=True)        

## HueSaturationValue

In [ ]:
aug = A.HueSaturationValue(hue_shift_limit=10,p = 1.0)
plot_img_2(df_oof, seq, frame, model_versions, use_clahe=False, aug=aug)
plot_img_2(df_oof, seq_tp, frame_tp, model_versions, use_clahe=False, aug=aug)        

## RGBShift

In [ ]:
aug = A.RGBShift(r_shift_limit = 40, b_shift_limit=0, g_shift_limit=0, p = 1.0)
plot_img_2(df_oof, seq, frame, model_versions, use_clahe=False, aug=aug)
plot_img_2(df_oof, seq_tp, frame_tp, model_versions, use_clahe=False, aug=aug)        

## FUnIE-GAN

In [ ]:
sys.path.append("./FUnIE-GAN/PyTorch/")
from nets import funiegan

model_path = "./FUnIE-GAN/PyTorch/models/funie_generator.pth"
model = funiegan.GeneratorFunieGAN()
model.load_state_dict(torch.load(model_path,map_location=torch.device('cpu')))

In [ ]:
model.eval()

In [ ]:
import torchvision.transforms as transforms
## data pipeline
img_width, img_height, channels = 1280, 1280, 3
transforms_ = [transforms.Resize((img_height, img_width), transforms.InterpolationMode.BICUBIC),
               transforms.ToTensor(),
               transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),]
transform = transforms.Compose(transforms_)

In [ ]:
path = df_oof.query("sequence == 53708 and video_frame == 8926").iloc[0]["image_path"]
path = df_oof.query("sequence == 53708 and video_frame == 9653").iloc[0]["image_path"]

In [ ]:
from PIL import Image
from torch.autograd import Variable
import torch
Tensor = torch.FloatTensor 
#path = '../../data/tensorflow-great-barrier-reef/train_images/video_0/8926.jpg'
inp_img = transform(Image.open(path))
inp_img = Variable(inp_img).type(Tensor).unsqueeze(0)
gen_img = model(inp_img)

In [ ]:
from torchvision.utils import save_image

In [ ]:
save_image(gen_img, "./test2.jpg", normalize=True)

In [ ]:
plt.imshow(gen_img.squeeze().data.permute(1, 2, 0).numpy())